In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
df = pd.read_csv('./dataset.csv')
df['Drained_after_socializing'] = pd.Series(map(lambda x: x == 'Yes', df['Drained_after_socializing']))
df['Stage_fear'] = pd.Series(map(lambda x: x == 'Yes', df['Stage_fear']))
df['Personality'] = pd.Series(map(lambda x: x == 'Extrovert', df['Personality']))
df

In [ ]:
X = df.drop('Personality', axis=1)
y = df['Personality']

classifier1 = SVC(C=2, kernel='linear')
classifier2 = SVC(C=2, kernel='poly')
classifier3 = SVC(C=2, kernel='rbf')
classifier4 = SVC(C=2, kernel='sigmoid')
# classifier5 = SVC(C=2, kernel='precomputed')

scores1 = cross_val_score(classifier1, X, y, cv=5)
scores2 = cross_val_score(classifier2, X, y, cv=5)
scores3 = cross_val_score(classifier3, X, y, cv=5)
scores4 = cross_val_score(classifier4, X, y, cv=5)
# scores5 = cross_val_score(classifier5, X, y, cv=5)

print(f"Classifer 1 - linear: {np.average(scores1)}")
print(f"Classifer 2 - poly: {np.average(scores2)}")
print(f"Classifer 3 - rbf: {np.average(scores3)}")
print(f"Classifer 4 - sigmoid: {np.average(scores4)}")
# print(f"Classifer 5 - precomputed: {np.average(scores5)}")

In [34]:
def avaliate(kernel: str, start=0.1, end=5):
    plot_x = []
    plot_y = []
    for i in np.linspace(start, end, 100):
        classifier = SVC(C=i, kernel=kernel)

        scores = cross_val_score(classifier, X, y, cv=5)
        score = np.average(scores)

        plot_x.append(i)
        plot_y.append(score)

    return plot_x, plot_y


In [ ]:
linear_x, linear_y = avaliate('linear')
poly_x, poly_y = avaliate('poly')
rbf_x, rbf_y = avaliate('rbf')
sigmoid_x, sigmoid_y = avaliate('sigmoid')

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(30, 10))
ax1.set_title('linear')
ax2.set_title('poly')
ax3.set_title('rbf')
ax4.set_title('sigmoid')

ax1.set_xlabel('C')
ax2.set_xlabel('C')
ax3.set_xlabel('C')
ax4.set_xlabel('C')

ax1.set_ylabel('Score')
ax2.set_ylabel('Score')
ax3.set_ylabel('Score')
ax4.set_ylabel('Score')

ax1.plot(linear_x, linear_y)
ax2.plot(poly_x, poly_y)
ax3.plot(rbf_x, rbf_y)
ax4.plot(sigmoid_x, sigmoid_y)

## Aqui, os graficos mostraram que o kernel "poly" desempenha melhor
## e seu desempenho parece ser crescente perto do C == 5, então
## será estudado individualmente abaixo

In [ ]:
poly_x, poly_y = avaliate('linear', start=4, end=8)

fig, ax = plt.subplots(1, 1, figsize=(30, 10))
ax.set_title('SVM com kernel do tipo linear')
ax.set_xlabel('C')
ax.set_ylabel('Score')
ax.plot(poly_x, poly_y)